# Contrôle de l'orbite d'un satellite

On assimile le satellite à un point matériel de masse $m$. Sa position $x=(x_1,x_2)$, dans un système de coordonnées plan centré sur la Terre, est donnée par la solution de l'équation de Newton:

$$
\ddot x(t) = - \mu \frac{x(t)}{|x(t)|^3} + \frac{F(t)}{m}
$$

où $F=(F_1,F_2)$ est la poussée et $\mu$ la constante gravitationnelle de la Terre. 

Nous allons commencer par tracer quelques trajectoires. Nous aurons besoin des modules `numpy` pour le calcul numérique, `matplotlib.pyplot` pour les graphiques, et de la fonction `odeint` du module `scipy.integrate` pour intégrer les systèmes différentiels. Nous définissons également quelques variables globales:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.integrate import odeint

p0 = 11.625 # Low Earth Orbit altitude
pf = 42.165 # Geostationary Orbit altitude
mu = 398600.47 * 10**(-9) * 3600**2 # Earth gravitation constant
alpha = np.sqrt(mu / pf**3) # rescaled constant
qb = p0/(1.75*pf) # Rescaled Low Earth Orbit altitude
Tfinal = 24
dt = 0.1# time step 

En adimensionnant le système afin que l'orbite géostationnaire soit à l'altitude 1, l'équation d'état s'écrit:

$$
\left\{
\begin{array}{rcl}
\dot q(t) &=& \eta(t) \\
\dot \eta(t) & = & -\alpha^2\displaystyle{\frac{q(t)}{|q(t)|^3}}
\end{array}
\right .
$$

où $q=(q_1,q_2)$ est la position et $\eta=(\eta_1,\eta_2)$ la vitesse du satellite. La fonction suivante implémente le champ de vecteur qui définit ce système dynamique&nbsp;:


In [ ]:
def motion_eq(q, t):
    """
    Motion equations for the satellite trajectory.
    
    Args:
        q[0], q[1]: position
        q[1], q[2]: velocity

    Returns:    
        Trajectory equations
    """
    q1,q2,eta1,eta2 = q[0],q[1],q[2],q[3]
    k =  alpha**2 / np.sqrt( q1**2 + q2**2 )**3
    return [eta1, 
            eta2, 
            - k * q1,  
            - k * q2]


## 1. Tracé de quelques trajectoires

On construit une liste contenant les paramètres de quelques trajectoires :

In [ ]:
#
test_case = []
# test_case format: ("label curve","style curve",q_initial,T_final_plot)
# where q_initial contains the initial position and velocity [q_1, q_2, eta_1, eta_2]
test_case.append(("Geostationary orbit","b",[0, 1, -alpha, 0],24))
test_case.append(("Circular Low Earth Orbit","g",[qb, 0, 0, alpha/np.sqrt(qb)],1.5))
test_case.append(("Elliptic Orbit 1","r",[-p0/(0.25*pf), 0, 0, -0.2*alpha/np.sqrt(p0/pf)],12))
test_case.append(("Elliptic Orbit 2","y",[p0/(1.75*pf), 0, 0, 1.25*alpha/np.sqrt(p0/(1.75*pf))],7))


On parcourt alors la liste des cas tests, on intègre l'équation d'état avec la fonction `odeint` et on trace les trajectoires et les énergies.

In [ ]:
fig1, ax = plt.subplots(1, 1, figsize=(10,10))
fig2, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18,12))
plt.rc('font',size=18)


for case in test_case:
    case_label,case_style,q0,T_final_plot = case[0],case[1],case[2],case[3]
    ts = np.linspace(0, Tfinal, int(Tfinal/dt)+1)

    # Differential equation solution
    qs = odeint(motion_eq, q0, ts)
    
    # Plot of the trajectories
    position_x,position_y = qs[:,0],qs[:,1]
    vitesse_x,vitesse_y = qs[:,2],qs[:,3]
    ax.set_title("Various trajectories")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_xlim(-1.3,1.5)
    ax.set_ylim(-1.2,1.7)
    n_plot = (int)(T_final_plot/dt)+1
    ax.plot(position_x[0:n_plot], position_y[0:n_plot],case_style+"+",label=case_label)
    ax.legend()

    # Plot of the energies
    Ec = 0.5 * (vitesse_x * vitesse_x + vitesse_y * vitesse_y)
    Ep = - alpha**2 / np.sqrt(position_x * position_x + position_y * position_y)    
    ax1.set_title("Energies")
    ax1.set_ylabel("Kinetic energy")
    ax1.plot(ts,Ec,case_style+'-')
    ax2.set_ylabel("Potential energy")
    ax2.plot(ts,Ep,case_style+'-')
    ax3.set_ylabel("Total energy")
    ax3.plot(ts,Ec+Ep,case_style+'-')
    ax3.set_xlabel("Time")


## 2. Transfert en poussée minimale 

On veut passer d'une orbite basse à une orbite géostationnaire en deux manoeuvres impulsionnelles ([transfert de Hohman](https:\\fr.wikipedia.org\wiki\Orbite_de_transfert)), c'est-à-dire en imposant deux changements de vitesse, $\Delta V_0$ à l'instant initial, et $\Delta V_f$ à l'instant final $t_f$. 

On veut donc que la vitesse initiale soit $\eta(0) = \eta_0 + \Delta V_0$, et qu'au temps $t_f$, on soit sur une orbite géostationnaire, ce qui se traduit par $|q(t_f)|=1$ et $\eta(t_f) + \Delta V_f = \alpha {\cal R}q(t_f)$, où $\cal R$ est la rotation d'angle $\pi/2$. Le but est de minimiser $|\Delta V_0|^2 + |\Delta V_f|^2$. 

On cherche donc à résoudre le problème de minimisation:
$$
\min_{u_0\in{\mathbb R}^2} J(u_0) = \min_{u_0\in{\mathbb R}^2} \frac{1}{2}\left( |u_0-\eta_0 |^2 + |\eta(t_f) - \alpha{\cal R}q(t_f)|^2 \right ),
$$
sous la contrainte que $(q,\eta$) vérifie les équations d'état sur $[0,t_f]$, avec comme donnée initiale $(q_0,u_0)$, tout en atteignant la **cible** $|q(t_f)|=1$. Noter que l'instant final $t_f$ fait partie des inconnues.

Commençons par tracer les orbites de départ et d'arrivée, qui correspondent aux paramètres suivants: 

|Orbite           |Position initiale|&nbsp;&nbsp;Vitesse initiale&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
|---------------|---------------|----------------------------------------|
|Géostationnaire  |$q(0)=(0,1)$     | $\eta(0)=(-\alpha,0)$                    |
|Circulaire basse |$q(0)=(q_b,0)$   | $\eta(0)=\left(0,\frac{\alpha}{\sqrt{q_b}}\right)$         |




In [ ]:
q1_0, q2_0 =  qb, 0
q1dot_0, q2dot_0= 0, alpha/np.sqrt(qb)
test_case = []
# test_case format: ("label curve","style curve",q_initial,T_final_plot)
test_case.append(("Geostationary orbit","b",[0, 1, -alpha, 0],24))
test_case.append(("Circular Low Earth Orbit","g",[q1_0,q2_0,q1dot_0,q2dot_0],1.5))


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))
plt.rc('font',size=18)

for case in test_case:
    case_label,case_style,q0,T_final_plot = case[0],case[1],case[2],case[3]
    ts = np.linspace(0, Tfinal, int(Tfinal/dt)+1)

    # ODE solution
    qs = odeint(motion_eq, q0, ts)
    
    # Plot of the orbit
    position_x,position_y = qs[:,0],qs[:,1]
    vitesse_x,vitesse_y = qs[:,2],qs[:,3]
    ax.set_title("A Low Earth Orbit and a Geostationary Orbit")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_xlim(-1.3,1.3)
    ax.set_ylim(-1.2,1.6)
    n_plot = (int)(T_final_plot/dt)+1
    ax.plot(position_x[0:n_plot], position_y[0:n_plot],case_style+"+",label=case_label)
    ax.legend()


### État adjoint

On introduit l'**état adjoint** $p(t)=(s(t),\mu(t))\in{\mathbb R}^4$ solution de $\dot p = -A(q,\eta)^T p$ où

$$
A = \begin{bmatrix} 
0_{{\mathbb R}^{2\times 2}} & I_{{\mathbb R}^{2\times 2}} \\ 
\Gamma & 0_{{\mathbb R}^{2\times 2}}
\end{bmatrix}\in  {\mathbb R}^{4\times 4}
\qquad 
\Gamma = \frac{\alpha^2}{|q|^5}\begin{bmatrix} 
2 q_1^2 - q_2^2 & 3 q_1 q_2 \\ 
 3 q_1 q_2 & 2 q_2^2 - q_1^2 
\end{bmatrix}\in  {\mathbb R}^{2\times 2}
$$

La fonction suivante définit le champ de vecteurs de $\mathbb R^8$ du système d'équations différentielles satisfait par l'état $y(t)=(q(t),\eta(t))$ et l'état adjoint $p(t) =(s(t),\mu(t))$:

In [ ]:
def motion_adjoint_eq_minimum_thrust(q, t):
    """
    Motion equations for the satellite trajectory
    and adjoint equations for the minimum thrust problem.
    
    Args:
        q[0], q[1]: position (q)
        q[2], q[3]: velocity (eta)
        q[4], q[5]: adjoint position (s) 
        q[6], q[7]: adjoint velocity (mu)
    
    Returns:
        Motion and adjoint equations.
    """
    q1,q2,eta1,eta2 = q[0],q[1],q[2],q[3]
    s1,s2,mu1,mu2 = q[4],q[5],q[6],q[7]
    tmp = np.sqrt( q1**2 + q2**2 )
    k1 =  alpha**2 / tmp**3
    k2 =  alpha**2 / tmp**5
    return [eta1, 
            eta2, 
            - k1 * q1,  
            - k1 * q2,
            - k2 * ((2*q1**2 - q2**2)*mu1 + (3*q1*q2        )*mu2),
            - k2 * ((3*q1*q2        )*mu1 + (2*q2**2 - q1**2)*mu2),
            - s1,
            - s2
            ]



## Méthode de tir

Le principe d'une **méthode de tir** est de chercher de manière itérative les conditions initiales sur l'état et l'état adjoint qui permettent de vérifier les conditions à l'instant final.

Ici, la condition initiale sur l'état est donnée par $(q_0,\eta_0)$, où $q_0$ est connue et $\eta_0 = u_0$ est inconnue (c'est le contrôle). La condition initiale sur l'état adjoint est $(s_0,\mu_0)$, où $s_0$ est inconnue et $\mu_0=\eta_0 - u_0$ d'après la condition d'optimalité. Ainsi la donnée des cinq scalaires $u_0\in{\mathbb R}^2$, $s_0\in{\mathbb R}^2$ et $t_f$ permet de résoudre le système différentiel vérifié par l'état et l'état adjoint sur $[0,t_f]$. 

À l'instant final $t_f$, les variables d'état $(q,\eta)$ et adjointes $(s,\mu)$ vérifient les relations suivantes:

- Atteinte de la cible :
$$
|q(t_f)|^2 - 1 = 0
$$

- Conditions de transversalité :
\begin{align}
-\mu(t_f) + \eta(t_f) - \alpha {\cal R} q(t_f) & = 0, \\
( - s(t_f) + \alpha {\cal R}\eta(t_f) , \cal R q(t_f))_{{\mathbb R}^2} & = 0, \\
\end{align}

- Annulation du hamiltonien :
$$
( s(t_f), \eta(t_f))_{{\mathbb R}^2}  - \alpha^2 ( \mu(t_f), q(t_f))_{{\mathbb R}^2}   = 0.
$$

La fonction python suivante permet d'évaluer
$$
F_p(u_0,s_0,t_f) 
= 
\begin{pmatrix}
|q(t_f)|^2 - 1\\
-\mu(t_f) + \eta(t_f) - \alpha {\cal R} q(t_f) \\
( - s(t_f) + \alpha {\cal R}\eta(t_f) , \cal R q(t_f))_{{\mathbb R}^2} \\
( s(t_f), \eta(t_f))_{{\mathbb R}^2}  - \alpha^2 ( \mu(t_f), q(t_f))_{{\mathbb R}^2} \end{pmatrix}
$$

Noter que chaque évaluation de $F_p$ nécessite une résolution complète du système différentiel état + adjoint. 

In [ ]:
def Fp_minimum_thrust(X):
    """
    Function to cancel (shooting method) for the minimum thrust problem.
    
    Args:
        X = [eta1_init, eta2_init, s1_init, s2_init, tf]

    Return:
        The 5 quantities to cancel.
    """
    global nb_eval,resultats
    t1 = np.linspace(0,X[4],int(X[4]/dt)+1)
    # The equation is integrated until the final time tf
    qs = odeint(motion_adjoint_eq_minimum_thrust, 
        [q1_0, # initial position (q1)
        q2_0,  # initial position (q2)
        X[0],  # initial velocity (eta1 = u0_1 = control)
        X[1],  # initial velocity (eta2 = u0_2 = control)
        X[2],  # initial adjoint position (s1)
        X[3],  # initial adjoint position (s2)
        -X[0]+q1dot_0, # initial adjoint velocity (mu1), q1dot_0 = eta0_1 (given original velocity of the trajectory)
        -X[1]+q2dot_0],# initial adjoint velocity (mu2), q2dot_0 = eta0_2 (given original velocity of the trajectory)
                       # given by the optimality condition
        t1)
    # qs is a np.array with 8 columns and as many rows as time steps 
    # The last row is extracted to get the solution at time t=tf
    q1,q2,eta1,eta2,s1,s2,mu1,mu2 = qs[-1,:] 
    Rq1,Rq2 = -q2,q1
    Reta1,Reta2 = -eta2,eta1
    tmp = np.sqrt( q1**2 + q2**2 )
    k1 =  alpha**2 / tmp**3
    XZero = [0,0,0,0,0]
    XZero[0] = q1**2 + q2**2 - 1
    XZero[1] = -mu1 + eta1 - alpha * Rq1
    XZero[2] = -mu2 + eta2 - alpha * Rq2
    XZero[3] = (-s1 + alpha*Reta1)*Rq1 + (-s2 + alpha*Reta2)*Rq2
    XZero[4] = s1*eta1 + s2*eta2 - k1 * (mu1*q1 + mu2*q2)
    
    # For the post-processing, the following data are stored:
    # iteration number; trajectory (q1,q2); cost value; X
    Jval = 0.5 * ( (q1dot_0 - X[0])**2 + (q2dot_0 - X[1])**2 ) + 0.5 * ( (eta1 - alpha*Rq1)**2 + (eta2 - alpha*Rq2)**2 )
    nb_eval = nb_eval + 1
    resultats.append((nb_eval,qs[:,0],qs[:,1],Jval,X[0],X[1],X[2],X[3],X[4]))
    return XZero
            


Le but de la méthode de tir est de chercher $(u_0,s_0,t_f)\in{\mathbb R}^5$ tel que

$$
F_p(u_0,s_0,t_f) = 0
$$

On va utiliser pour cela la fonction `fsolve`, du module `scipy.optimize`, qui permet de trouver un zéro d'une fonction.

In [ ]:
from scipy.optimize import fsolve

On pourra vérifier que l'initialisation de la méthode de tir de tir est très délicate. 

In [ ]:
u0_1, u0_2 = -0.6, 0.6
s0_1, s0_2 = 1., 1.
tf = 1.
nb_eval = 0 # how many times the state and adjoint equations are solved (postprocessing)
resultats = [] # to store the intermediate results at each shooting iteration (postprocessing)

a = fsolve(Fp_minimum_thrust,[u0_1,u0_2,s0_1,s0_2,tf])


## Affichage des résultats

On trace l'évolution de la fonction coût $J$, et des 5 inconnues $\eta_1(0) = u_{0,1},\eta_2(0)= u_{0,2},s_1(0),s_2(0),t_f$ au cours des itérations de la méthode de tir:

In [ ]:
# Plot of the cost function and shooting variables
fig1,(ax1,ax2,ax3) = plt.subplots(3, 1,figsize=(16, 10))
fig2,(ax4,ax5,ax6) = plt.subplots(3, 1,figsize=(16, 10))
plt.rc('font',size=18)

Jval = []
eta1val = []
eta2val = []
s1val = []
s2val = []
tfval = []

for iter,position_x,position_y,Jv,eta1,eta2,s1,s2,tf in resultats:
    Jval.append(Jv)
    eta1val.append(eta1)
    eta2val.append(eta2)
    s1val.append(s1)
    s2val.append(s2)
    tfval.append(tf)

ax1.set_ylabel("Cost function")
ax1.plot(Jval,'r')

ax2.set_ylabel("Final time")
ax2.plot(tfval,'g')

ax3.set_ylabel("eta1")
ax3.plot(eta1val,'b')


ax4.set_ylabel("eta2")
ax4.plot(eta2val,'b')

ax5.set_ylabel("s1")
ax5.plot(s1val,'y')

ax6.set_ylabel("s2")
ax6.plot(s2val,'y')

plt.show()

On trace la trajectoire initiale et celle obtenue à convergence:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[12,12])
plt.rc('font',size=18)
#
# Plot of the two orbits (initial and target)
#
for case in test_case:
    case_label,case_style,q0,T_final_plot = case[0],case[1],case[2],case[3]
    ts = np.linspace(0, Tfinal, int(Tfinal/dt)+1)

    # ODE solution 
    qs = odeint(motion_eq, q0, ts)
    
    # Plot of the orbit
    position_x,position_y = qs[:,0],qs[:,1]
    vitesse_x,vitesse_y = qs[:,2],qs[:,3]
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_xlim(-1.3,1.3)
    ax.set_ylim(-1.2,1.5)
    n_plot = (int)(T_final_plot/dt)+1
    ax.plot(position_x[0:n_plot], position_y[0:n_plot],case_style+"-",label=case_label)

# Plot of the trajectory corresponding to the initial parameters given to the shooting algorithm

iter,position_x,position_y,Jv,eta1,eta2,s1,s2,tf = resultats[0]
ax.plot(position_x,position_y,'orange',label='Initial guess')

# Plot of the optimal trajectory (last iteration of the shooting algorithm)

iter,position_x,position_y,Jv,eta1,eta2,s1,s2,tf = resultats[-1]
ax.plot(position_x,position_y,'red',label='Optimal trajectory')

ax.set_title("Optimal thrust trajectory from a Low Earth to a Geostationary Orbit")
ax.legend()

plt.show()


Animation des trajectoires successives obtenues par l'algorithme de tir. **N.B.:** nécessite que la librairie `ffmpeg` soit installée sur votre ordinateur.

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

fig, ax = plt.subplots(1, 1, figsize=[12,12])
plt.rc('font',size=18)
#
# Plot of the two orbits (initial and target)
#
for case in test_case:
    case_label,case_style,q0,T_final_plot = case[0],case[1],case[2],case[3]
    ts = np.linspace(0, Tfinal, int(Tfinal/dt)+1)

    # ODE solution 
    qs = odeint(motion_eq, q0, ts)
    
    # Plot of the orbit
    position_x,position_y = qs[:,0],qs[:,1]
    vitesse_x,vitesse_y = qs[:,2],qs[:,3]
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_xlim(-1.3,1.3)
    ax.set_ylim(-1.2,1.5)
    n_plot = (int)(T_final_plot/dt)+1
    ax.plot(position_x[0:n_plot], position_y[0:n_plot],case_style+"-",label=case_label)


line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = resultats[i-1][1]
    y = resultats[i-1][2]
    line.set_data(x, y)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

In [ ]:
HTML(anim.to_jshtml())